In [22]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import chromadb
import uuid
from langchain.chains import RetrievalQA
load_dotenv()
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import re

In [18]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage

In [ ]:
pip install sentence-transformers

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [15]:
llm=ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="deepseek-r1-distill-llama-70b"
)

In [4]:
response=llm.invoke("who are you")
response

AIMessage(content="<think>\n\n</think>\n\nGreetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 6, 'total_tokens': 50, 'completion_time': 0.16, 'prompt_time': 0.003498289, 'queue_time': 0.23289115700000002, 'total_time': 0.163498289}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_492bd52206', 'finish_reason': 'stop', 'logprobs': None}, id='run-84b5518b-a927-4d1a-bf40-0b7cb535cafd-0', usage_metadata={'input_tokens': 6, 'output_tokens': 44, 'total_tokens': 50})

In [ ]:
'''## Pdf reader

loader=PyPDFLoader("B09221-eng.pdf")
docs=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
# documents=documents[:5]

len(documents)


# Initialize embeddings model
embedding_model = OllamaEmbeddings(model="llama3.2:1b")

# Create Chroma vector database
vectorstore = Chroma.from_documents(documents, embedding_model, persist_directory="chroma_db")

# Save the database
vectorstore.persist()

# Load the vector store
vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embedding_model)

query = "what is the symptom of Ebola"
# results = vectorstore.similarity_search(query, k=3) 
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2}) 
results = retriever.get_relevant_documents(query)

for doc in results:
    print(doc.page_content)


response=llm.invoke("what is symptom of Ebola")
print(response)

context = " ".join([doc.page_content for doc in results])
prompt = f"Based on the following information, answer the question:\n\n{context}\n\nQuestion: {query}"
response = llm.invoke(prompt)
'''

74

In [ ]:
loader=PyPDFLoader("B09221-eng.pdf")
docs=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)



# Create Chroma vector database
vectorstore = Chroma.from_documents(documents, embedding_model, persist_directory="chroma-BAAI")

# Save the database
vectorstore.persist()

# Load the vector store
vectorstore = Chroma(persist_directory="chroma-BAAI", embedding_function=embedding_model)


C:\Users\dheeraj\AppData\Local\Temp\ipykernel_6172\362889348.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
C:\Users\dheeraj\AppData\Local\Temp\ipykernel_6172\362889348.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="chroma-BAAI", embedding_function=embedding_model)


In [11]:
query = "what is WHO"
results = vectorstore.similarity_search(query, k=3) 
# print(results)
for doc in results:
    print(doc.page_content)

Health Security Agency, United Kingdom of Great Britain and Northern Ireland 
6. WHO Collaborating Centre for Viral Hemorrhagic Fevers, Centers for Disease Control and Prevention, 
United States of America
Security Agency, United Kingdom of Great Britain and Northern Ireland), Fahn Taweh (National Public Health 
Institute, Liberia), Babs Verstrepen (Erasmus University Medical Center, Netherlands), Dan Youkee  (Kings 
College London, United Kingdom of Great Britain and Northern Ireland). 
WHO Headquarters and Regional Office staff contributors and reviewers:  
Mustafa Abdelaziz Aboualy (WHO Regional Office for the Eastern Mediterranean), Rachel Achilla ( WHO 
Regional Office for Africa), Jeffrey Gilbert (Emerging Diseases and Zoonoses, Geneva, Switzerland), Hieronyma 
Nelisiwe Gumede -Moeletsi ( WHO Regional Office for Africa ), Jean-Michel Heraud  ( Global Influenza 
Programme, Geneva, Switzerland), Jean de Dieu Iragena ( WHO Regional Office for Africa),  Kazunobu Kojima 
(Biosecurity 

In [ ]:
context = " ".join([doc.page_content for doc in results])
prompt = f"Based on the following information, answer the question:\n\n{context}\n\nQuestion: {query}"
response = llm.invoke(prompt)
print(response.content)

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5}),
    memory=memory,
)




In [24]:
chat_history = []
while True:
    query = input("Enter your question (or type 'exit' to quit): ")
    
    if query.lower() == "exit":
        break
    
    # Invoke the QA chain
    response = qa_chain.invoke({"question": query, "chat_history": chat_history})
    
    # Extract and clean response
    cleaned_response = re.sub(r"<think>.*?</think>", "", response["answer"], flags=re.DOTALL)
    
    # Update chat history
    chat_history.append(HumanMessage(query))
    chat_history.append(AIMessage(cleaned_response))

    # Display response
    print("\n🤖 Bot:", cleaned_response)



🤖 Bot: 

WHO stands for the World Health Organization. It is a specialized agency of the United Nations responsible for international public health. It is concerned with global health matters, particularly infectious diseases like Ebola and Marburg, and works to develop and implement policies that promote health, keep the world safe, and serve the vulnerable.

🤖 Bot: 

How are you?

🤖 Bot: 

LLM in this context likely stands for a specific program or initiative within the World Health Organization, possibly related to laboratory strengthening or logistical management, which are crucial for addressing public health emergencies like Ebola and Marburg.

🤖 Bot: <think>
